## Bias Correction

Jenna Ruzekowicz (jenna.ruzekowicz@nrel.gov) and Caleb Phillips (caleb.phillips@nrel.gov)

This notebook will read resource data and reference observation data and use it to compute a bias corrected version of the resource data (by multiple linear regression).













